<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [6]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import numpy as np
from transformers import pipeline
import time
import datetime
import os

In [9]:
roberta = pipeline(task='sentiment-analysis', 
                   model="mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis",
                   return_all_scores=True,
                   max_length=514, 
                   truncation=True)

In [21]:
def summary_stats(score):
    avg = round(np.mean(score),2)
    med = round(np.median(score),2)
    min = round(np.min(score),2)
    max = round(np.max(score),2)
    return(avg, med, min, max)

In [59]:
URL = "https://finance.yahoo.com/news/fintech-stocks-fall-amid-report-of-apples-financial-services-plans-201032568.html"
column_names = ["url", "pos_mean", "pos_median", "pos_min", "pos_max", "neg_mean", "neg_median", "neg_min", "neg_max", "neu_mean", "neu_median", "neu_min", "neu_max", "net_mean", "net_median", "net_min", "net_max"]
y_df = pd.DataFrame(columns = column_names)
y_df.loc[y_df.shape[0]] = [None] * len(column_names)
y_df.url = URL
y_df

,url,pos_mean,pos_median,pos_min,pos_max,neg_mean,neg_median,neg_min,neg_max,neu_mean,neu_median,neu_min,neu_max,net_mean,net_median,net_min,net_max
0,https://finance.yahoo.com/news/fintech-stocks-...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [66]:

def yahoo_article(URL,df):
    nf = "Content is currently unavailable"
    page = requests.get(URL,headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'})
    soup = BeautifulSoup(page.content, "html.parser")
    if nf in soup.prettify():
        pass
    try:
        try:
            master = soup.find(id="Masterwrap")
            final = master.find("div", class_="caas-body")
        except AttributeError:
            master = soup.find(id="Masterwrap2Col")
            final = master.find("div", class_="caas-body")
    except AttributeError:
        pass
    text = re.sub('(<p>)', ' ', str(final))
    text = re.sub('<[^>]+>', '', str(text))
    sen_list = text.split(". ")[:-2]
    article = [".\n ".join(sen_list[:-2])][0] # remove author signature
    
    pos = []
    neg = []
    neu = []
    net = []
    #column_names = ["pos_mean", "pos_median", "pos_min", "pos_max", "neg_mean", "neg_median", "neg_min", "neg_max", "neu_mean", "neu_median", "neu_min", "neu_max", "net_mean", "net_median", "net_min", "net_max"]
    #df = pd.DataFrame(columns = column_names)

    for sen in sen_list:
        output = roberta(sen)
        print(sen)
        print("negative: " + str(output[0][0]['score']))
        print("neutral: " + str(output[0][1]['score']))
        print("positive: " + str(output[0][2]['score']))
        print()
        neg.append(output[0][0]['score'])
        neu.append(output[0][1]['score'])
        pos.append(output[0][2]['score'])
        net.append(pos[-1] - neg[-1])
    
    column_names = ["sentence", "positive", "negative", "neutral"]
    s_df = pd.DataFrame(columns = column_names)
    s_df.sentence = sen_list
    s_df.positive = pos
    s_df.negative = neg
    s_df.neutral = neu

    df.pos_mean, df.pos_median, df.pos_min, df.pos_max = summary_stats(pos)
    df.neg_mean, df.neg_median, df.neg_min, df.neg_max = summary_stats(neg)
    df.neu_mean, df.neu_median, df.neu_min, df.neu_max = summary_stats(neu)
    df.net_mean, df.net_median, df.net_min, df.net_max = summary_stats(net)
    return(df, s_df)

In [67]:
total_df, sen_df = yahoo_article(URL,y_df)

 Block (SQ), PayPal (PYPL), Marqeta (MQ) Visa (V), and other payment companies' shares fell on Wednesday afternoon after Bloomberg reported Apple (AAPL) is working on a plan dubbed “Breakout” aimed at bringing financial services in-house
negative: 0.9982985854148865
neutral: 0.0012821536511182785
positive: 0.00041928407154046

The report states the tech giant is building a payment processing technology and infrastructure as part of an effort to decrease its dependence on outside partners
negative: 0.0002527672331780195
neutral: 0.002566627925261855
positive: 0.9971805810928345

The strategy is aimed at future financial products instead of existing ones
negative: 6.807414320064709e-05
neutral: 0.9998821020126343
positive: 4.977865319233388e-05

Shares of Block, formally known as Square, fell by more than 4% an hour before the closing bell on Wednesday
negative: 0.9983087778091431
neutral: 0.0012527001090347767
positive: 0.0004385588981676847

PayPal shares were down about 2%
negative: 0

In [73]:
total_df.to_csv("tot.csv", index=False)

In [74]:
sen_df.to_csv("sen.csv", index=False)